# pandapower Optimal Power Flow
This is an introduction into the usage of the pandapower optimal power flow. It shows how to set the constraints and the cost factors into the pandapower element tables.

## Example Network

We use the following four bus example network for this tutorial:

<img src="pics/example_opf.png" width="50%">

We first create this network in pandapower:

In [193]:
import pandapower as pp
import numpy as np
import math as mt
import sympy as sy
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net, vn_kv=220.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)

#create 220/110 kV transformer
pp.create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV")
net.trafo.vkr_percent.at[0] = 0
net.trafo.pfe_kw.at[0]=0

#create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')

#create loads
pp.create_load(net, bus2, p_mw=60, controllable=False)
pp.create_load(net, bus3, p_mw=70, controllable=False)
pp.create_load(net, bus4, p_mw=10, controllable=False)

#create generators
eg = pp.create_ext_grid(net, bus1, min_p_mw=-1000, max_p_mw=1000)
g0 = pp.create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80,  vm_pu=1.01, controllable=True)
g1 = pp.create_gen(net, bus4, p_mw=100, min_p_mw=0, max_p_mw=100, vm_pu=1.01, controllable=True)

In [194]:
net.trafo

,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,...,tap_neutral,tap_min,tap_max,tap_step_percent,tap_step_degree,tap_pos,tap_phase_shifter,parallel,df,in_service
0,None,100 MVA 220/110 kV,0,1,100.0,220.0,110.0,12.0,0.0,0.0,...,0,-9,9,1.5,0.0,0,False,1,1.0,True


## Loss Minimization

We specify the same costs for the power at the external grid and all generators to minimize the overall power feed in. This equals an overall loss minimization:

In [195]:
costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=10)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

We run an OPF:

In [196]:
pp.runopp(net, delta=1e-16)

This function runs an Optimal Power Flow using the PYPOWER OPF. To make sure that the PYPOWER OPF converges, we decrease the power tolerance `delta` (the default value is `delta=1e-10`). The power tolerance `delta` is a measure of the extent to which exceeding of minimum and maximum power limits is tolerated. That is, in above case, the limits considered by the OPF for the generators are `min_p_mw - delta` and `max_p_mw + delta` as lower and upper bound respectively on the active power. 

Let's check the results:

In [197]:
net.res_ext_grid

,p_mw,q_mvar
0,59.999633,2.55991


In [198]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,69.997330,-1.993949,-4.130836,0.999560
1,10.003033,-1.495470,-4.130649,0.999561


Since all costs were specified the same, the OPF minimizes overall power generation, which is equal to a loss minimization in the network. The loads at buses 3 and 4 are supplied by generators at the same bus, the load at Bus 2 is provided by a combination of the other generators so that the power transmission leads to minimal losses.

## Individual Generator Costs

Let's now assign individual costs to each generator.

We assign a cost of 10 ct/kW for the external grid, 15 ct/kw for the generator g0 and 12 ct/kw for generator g1:

In [199]:
net.poly_cost.cp1_eur_per_mw.at[costeg] = 10
net.poly_cost.cp1_eur_per_mw.at[costgen1] = 15
net.poly_cost.cp1_eur_per_mw.at[costgen2] = 12

And now run an OPF:

In [200]:
pp.runopp(net, delta=1e-16)

We can see that all active power is provided by the external grid: 

In [201]:
net.res_ext_grid

,p_mw,q_mvar
0,143.925799,9.608184


In [202]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000075,8.346090,-16.315952,0.970337
1,0.000205,10.176077,-13.387207,0.992411


This makes sense, because the external grid has the lowest cost of all generators and we did not define any constraints.

The dispatch costs are given in net.res_cost:

In [203]:
net.res_cost

1439.26157143092

### Transformer Constraint

Since all active power comes from the external grid and subsequently flows through the transformer, the transformer is overloaded with a loading of about 145%:

In [204]:
net.res_trafo.loading_percent

0    144.247362
Name: loading_percent, dtype: float64

# Adding all the equality constraints into the optimum power flow.

In [205]:
def constraints(loading_trafo,loading_line,min_bus_voltage_pu,max_bus_volatge_pu ):

    net.trafo["max_loading_percent"]=loading_trafo
    net.line["max_loading_percent"]=loading_line
    net.bus["min_vm_pu"] = min_bus_voltage_pu
    net.bus["max_vm_pu"] = max_bus_volatge_pu
    
    


In [206]:
constraints(loading_trafo=70,loading_line=70,min_bus_voltage_pu=0.98,max_bus_volatge_pu=1.02)
net.trafo


,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,...,tap_min,tap_max,tap_step_percent,tap_step_degree,tap_pos,tap_phase_shifter,parallel,df,in_service,max_loading_percent
0,None,100 MVA 220/110 kV,0,1,100.0,220.0,110.0,12.0,0.0,0.0,...,-9,9,1.5,0.0,0,False,1,1.0,True,70


We now limit the transformer loading to 70%:

In [207]:
#net.trafo["max_loading_percent"] = 50

(the max_loading_percent parameter can also be specified directly when creating the transformer)
and run the OPF:

In [208]:
pp.runopp(net, delta=1e-16)

We can see that the transformer complies with the maximum loading:

In [209]:
net.res_trafo.loading_percent

0    70.000134
Name: loading_percent, dtype: float64

And power generation is now split between the external grid and generator 1 (which is the second cheapest generation unit):

In [210]:
net.res_ext_grid

,p_mw,q_mvar
0,69.981986,-1.419195


In [211]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000021,3.990395,-8.341662,0.983754
1,72.602657,3.494350,-3.790424,1.019834


This comes of course with an increase in dispatch costs:

In [212]:
net.res_cost

1571.0520605183985

### Line Loading Constraints

Wen now look at the line loadings:

In [214]:
net.res_line.loading_percent

0    27.894144
1    52.026768
2    17.271836
Name: loading_percent, dtype: float64

and run the OPF with a 50% loading constraint:

In [215]:
#net.line["max_loading_percent"] = 50
pp.runopp(net, delta=1e-16)

Now the line loading constraint is complied with:

In [216]:
net.res_line.loading_percent

0    27.896828
1    52.034176
2    17.253980
Name: loading_percent, dtype: float64

And all generators are involved in supplying the loads:

In [217]:
net.res_ext_grid

,p_mw,q_mvar
0,69.983764,-1.34463


In [218]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000045,4.013317,-8.342433,0.983634
1,72.601171,3.398851,-3.788480,1.019663


This of course comes with a once again rising dispatch cost:

In [219]:
net.res_cost

1571.052378504202

### Voltage Constraints

Finally, we have a look at the bus voltage:

In [220]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.000000,0.000000,-69.983764,1.344630,10.000000,-4.452121e-22
1,1.005164,-4.792681,60.000000,0.000000,12.212110,2.352639e-02
2,0.983634,-8.342433,69.999955,-4.013317,12.940276,6.372106e-22
3,1.019663,-3.788480,-62.601171,-3.398851,12.000002,9.450153e-22


and constrain it:

In [221]:
net.bus["min_vm_pu"] = 1.0
net.bus["max_vm_pu"] = 1.02
pp.runopp(net, delta=1e-16)

We can see that all voltages are within the voltage band:

In [222]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.000000,0.000000,-69.863306,4.271737,10.000000,-2.448771e-22
1,1.008652,-4.767829,60.000000,0.000000,12.220574,2.775923e-03
2,1.000000,-8.584930,69.999986,-18.827574,13.016755,1.076081e-21
3,1.020000,-3.673647,-62.822507,8.353834,12.000001,-4.761634e-22


And all generators are once again involved in supplying the loads:

In [223]:
net.res_ext_grid

,p_mw,q_mvar
0,69.863306,-4.271737


In [224]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000014,18.827574,-8.584930,1.00
1,72.822507,-8.353834,-3.673647,1.02


This of course comes once again with rising dispatch costs:

In [225]:
net.res_cost

1572.5033636832097

In [226]:
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,max_loading_percent
0,None,149-AL1/24-ST1A 110.0,1,2,70.0,0.194,0.41,8.75,0.0,0.47,1.0,1,ol,True,70
1,None,149-AL1/24-ST1A 110.0,2,3,50.0,0.194,0.41,8.75,0.0,0.47,1.0,1,ol,True,70
2,None,149-AL1/24-ST1A 110.0,3,1,40.0,0.194,0.41,8.75,0.0,0.47,1.0,1,ol,True,100


In [227]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,24.918092,-8.352159,-24.176463,7.570974,0.741629,-0.781185,0.136754,0.132970,0.136754,1.008652,-4.767829,1.000000,-8.584930,29.096621
1,-45.823522,11.256600,47.623969,-9.148208,1.800446,2.108392,0.247662,0.249540,0.249540,1.000000,-8.584930,1.020000,-3.673647,53.093661
2,15.198538,0.794373,-15.054786,-1.859468,0.143752,-1.065095,0.078314,0.078935,0.078935,1.020000,-3.673647,1.008652,-4.767829,16.794596


In [228]:
#Line Impedances
z23 = mt.sqrt((70*0.194)**2 + (70*0.41**2 )) # line impedance from bus 2 to bus 3
z34 = mt.sqrt((50*0.194)**2 + (50*0.41**2 )) # line impedance from bus 3 to bus 4
z24 = mt.sqrt((40*0.194)**2 + (40*0.41**2 )) # line impedance from bus 2 to bus 4




LMP for Bus 1

In [257]:
# Setting constrants on Line 2 (from Bus 2 to Bus 4) - Maximum loading 17%
dp3,dp4 = sy.symbols("dp3 dp4") # dp3 - Power change at Bus 3, dp4 Power change at Bus 4
expr1 = (dp3 + dp4 -1) # Bus3 and Bus 4 to supply 1 MW power to Bus 2.
# for Bus 4 to supply 1 MW to Bus 2, 75% power would flow from bus 4 to 2 and 25% power would flow from Bus 4 to Bus 3 to Bus 2.
# for Bus 3 to supply 1 MW to Bus 2, 56.25% power would flow from bus 3 to 2 and 43.75% power would flow from Bus 3 to Bus 4 to Bus 2.
expr2 = (0.75*dp4 + 0.4375*dp3)
sol = sy.solve((expr1,expr2),(dp3,dp4))
sol


{dp3: 2.40000000000000, dp4: -1.40000000000000}

dp3

## DC OPF

pandapower also provides the possibility of running a DC Optimal Power Flow:

In [230]:
pp.rundcopp(net, delta=1e-16)

Since voltage magnitudes are not included in the DC power flow formulation, voltage constraints cannot be considered in the DC OPF:

In [231]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.000000,0.000000,-69.999998,4.271737,10.0,0.0
1,1.008652,-4.812932,60.000000,0.000000,12.0,0.0
2,1.000000,-8.125492,69.999999,-18.827574,12.0,0.0
3,1.020000,-3.696611,-60.000001,8.353834,12.0,0.0


Line and transformer loading limits are however complied with:

In [232]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,24.374999,0.0,-24.374999,0.0,0.0,0.0,0.127936,0.127936,0.127936,1.0,-4.812932,1.0,-8.125492,27.220333
1,-45.625000,0.0,45.625000,0.0,0.0,0.0,0.239469,0.239469,0.239469,1.0,-8.125492,1.0,-3.696611,50.950882
2,14.375001,0.0,-14.375001,0.0,0.0,0.0,0.075449,0.075449,0.075449,1.0,-3.696611,1.0,-4.812932,16.053019


In [233]:
net.res_trafo

,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,pl_mw,ql_mvar,i_hv_ka,i_lv_ka,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree,loading_percent
0,69.999998,0.0,-69.999998,0.0,0.0,0.0,0.183702,0.367405,1.0,0.0,1.0,-4.812932,69.999998


As are generator limits:

In [234]:
net.gen

,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,in_service,slack_weight,type,controllable,min_p_mw,max_p_mw,power_station_trafo
0,None,2,80.0,1.01,NaN,NaN,NaN,1.0,False,True,0.0,None,True,0.0,80.0,NaN
1,None,3,100.0,1.01,NaN,NaN,NaN,1.0,False,True,0.0,None,True,0.0,100.0,NaN


In [235]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000001,18.827574,-8.125492,1.0
1,70.000001,-8.353834,-3.696611,1.0


The cost function is the same for the linearized OPF as for the non-linear one:

In [236]:
net.res_cost

1540.0000061866479

## Piecewise linear cost functions

The OPF also offers piecewise linear cost functions. Let us first check the actual cost function setup:

In [237]:
net.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0,ext_grid,0.0,10.0,0.0,0.0,0.0,0.0
1,0,gen,0.0,15.0,0.0,0.0,0.0,0.0
2,1,gen,0.0,12.0,0.0,0.0,0.0,0.0


An element can either have polynomial costs or piecewise linear costs at the same time. So let us first delete the polynomial costs in order to avoid confusion and errors:

In [238]:
net.poly_cost.drop(net.poly_cost.index.values, inplace=True)

The results above have been produced with linear polynomial cost functions. Let's try to reproduce the results using piecewise linear cost functions. Costs have to be defined for the whole range of generators and external grids:

In [239]:
net.gen[["min_p_mw", "max_p_mw"]]

,min_p_mw,max_p_mw
0,0.0,80.0
1,0.0,100.0


In [240]:
net.ext_grid[["min_p_mw", "max_p_mw"]]

,min_p_mw,max_p_mw
0,-1000.0,1000.0


We define the piecewise linear cost as constant over the whole range to reproduce the polyomial costs defined above:

In [241]:
pp.create_pwl_cost(net, 0, "gen", [[0, 80, 15]])
pp.create_pwl_cost(net, 1, "gen", [[0, 100, 12]])
pp.create_pwl_cost(net, 0, "ext_grid", [[-1000, 1000, 10]])

2

Let us check the results from the previous OPF again!

In [242]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000001,18.827574,-8.125492,1.0
1,70.000001,-8.353834,-3.696611,1.0


In [243]:
net.res_cost

1540.0000061866479

We run the same OPF now with different cost function setup. We should get the exact same results:

In [244]:
pp.rundcopp(net, delta=1e-16)

In [245]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000001,18.827574,-8.125492,1.0
1,70.000001,-8.353834,-3.696611,1.0


In [246]:
net.res_cost

1540.0000061866479

Now lets define real piecewise linear costs for generator 1. We define the costs as 12€/MW up to 70MW and 20€/MW from 70MW to 100MW:

In [247]:
net.pwl_cost.points.loc[1] = [[0, 70, 12], [70, 100, 20]]

And run another OPF:

In [248]:
pp.rundcopp(net, delta=1e-16)

Now we can see that generator 1 only dispatches 70MW, above which generator 0 becomes less expensive and is therefore dispatched:

In [249]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000010,18.827574,-8.125492,1.0
1,69.999994,-8.353834,-3.696611,1.0


## Debugging

For more information on the status of the OPF solver, set verbose=True:

In [250]:
pp.runopp(net, verbose=True, delta=1e-16)

PYPOWER Version 5.1.4, 27-June-2018 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!

Converged in 0.77 seconds
Objective Function Value = 1577.74 $/hr
| PyPower (ppci) System Summary - these are not valid for pandapower DataFrames|

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              4     Total Gen Capacity    1180.0       -3000000000.0 to 3000000000.0
Generators         3     On-line Capacity      1180.0       -3000000000.0 to 3000000000.0
Committed Gens     3     Generation (actual)    142.5               5.7
Loads              3     Load                   140.0               0.0
  Fixed            3       Fixed                140.0               0.0
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches           4   

In [251]:
from sympy import symbols, Eq, solve